In [2]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal, matrix_normal, norm

# Data generation

In [3]:
# setting
ControlList = ['InControl','OutControl']
DataTypeList = ['Profile','Image']
CovList = ['TriDiagonal','Exponential']
DistList = ['Normal','Exponential']
phiList = [0.3, 0.7]
rhoList = [0.3]

In [4]:
def getCov(rho, p, type):
    Sigma = np.eye(p)
    if type == CovList[0]:
        tmp = np.ones(p-1)*rho
        Sigma = Sigma+np.diag(tmp, k = 1)+np.diag(tmp, k = -1)
    elif type == CovList[1]:
        for d in range(1, p-1):
            tmp = np.ones(p-d)*(rho**d)
            Sigma = Sigma+np.diag(tmp, k = d)+np.diag(tmp, k = -d)
    return Sigma

## Type 1

### In-control

In [48]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 5
TrnLen = 200 # full scale is 50000
ARL0= 1000
mu0 = 5*np.ones(p)
ControlType = 'InControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,'phi',str(phi),'rho',str(rho))
TagJoin = ''.join(Tag)
print(Tag)
trn = []
Cov = getCov(rho,p,CovType)
Phi = phi * np.eye(p)   

eps0 = multivariate_normal.rvs(np.zeros(p), Cov)
for i in tqdm(range(TrnLen)):
    eps = np.matmul(Phi, eps0) + multivariate_normal.rvs(np.zeros(p), Cov)
    trn.append(mu0 + eps)
    eps0 = eps
trn = np.array(trn)
np.save("data/in-control/Trn"+TagJoin+".npy", trn)

('InControl', 'Profile', 'TriDiagonal', 'Normal', 'phi', '0.3', 'rho', '0.3')


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:06<00:00, 31.65it/s]


### Out-of-control

In [71]:
p = 200
w = 5
deltanorm = 2 # the norm of shift for Type1
ShiftType = 'Sparse'

if ShiftType == 'Sparse':
    shift_basis = np.zeros(p) # sparse
    shift_basis[18:23] = np.ones(5)
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis

elif ShiftType == 'Stepwise':
    shift_basis = np.zeros(p) # piece_1
    shift_basis[50:100] = np.ones(50)
    shift_basis[100:150] = 2 * np.ones(50)
    shift_basis[150:200] = 3 * np.ones(50)
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis

elif ShiftType == 'Zigzag':
    shift_basis = np.zeros(p) # zigzag
    left = 1
    right = -1
    for j in range(10):
        shift_basis[j*20:(j+1)*20] = np.linspace(left, right, 20)
        left, right = right, left
    shift_basis = deltanorm/np.sqrt(np.sum(shift_basis**2))*shift_basis     


In [72]:
seedid = 19940402
np.random.seed(seedid)
TrnLen = 1000 # length of sequence
SeqNum = 5 # number of sequence, full scale is 1000
ARL0 = 1000
mu0 = 5*np.ones(p)
mu1 = mu0+shift_basis
ControlType = 'OutControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,ShiftType,'phi',str(phi),'rho',str(rho),'deltanorm',str(deltanorm))
TagJoin = ''.join(Tag)
print(Tag)

Cov = getCov(rho,p,CovType)
Phi = phi * np.eye(p)   


trn = []
for n in tqdm(range(SeqNum)):
    trn_seq = []
    eps0 = multivariate_normal.rvs(np.zeros(p), Cov)
    for i in range(TrnLen):
        eps = np.matmul(Phi, eps0) + multivariate_normal.rvs(np.zeros(p), Cov)
        trn_seq.append(mu1 + eps)
        eps0 = eps
    trn.append(trn_seq)
trn = np.array(trn)

np.save("data/out-of-control/Trn"+TagJoin+".npy", trn)

('OutControl', 'Profile', 'TriDiagonal', 'Normal', 'Sparse', 'phi', '0.3', 'rho', '0.3', 'deltanorm', '2')


100%|██████████| 5/5 [02:26<00:00, 29.31s/it]


In [73]:
trn.shape #(numseq,lenseq,dimofprofile)

(5, 1000, 200)

## Type 2

### In-control

In [79]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 100
TrnLen = 200 # full scale is 30000
ARL0 = 1000

ControlType = 'InControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'

TagN = (ControlType,DataType,CovType,'Normal','phi',str(phi),'rho',str(rho))
TagNJoin = ''.join(TagN)
TagE = (ControlType,DataType,CovType,'Exponential','phi',str(phi),'rho',str(rho))
TagEJoin = ''.join(TagE)
print(CovType)
Covw = getCov(rho,w,CovType)
Covp = getCov(rho,p,CovType)

trnN = []
trnE = []
for i in tqdm(range(TrnLen)):
    eps = matrix_normal.rvs(mean=np.zeros([w,p]), rowcov=Covw, colcov=Covp)
    trnN.append(m + eps)
    trnE.append(m + (-1 * np.log(1 - norm.cdf(eps))))
trnN = np.array(trnN)
trnE = np.array(trnE)

np.save("data/in-control/Trn"+TagNJoin+".npy", trnN)
np.save("data/in-control/Trn"+TagEJoin+".npy", trnE)

TriDiagonal


100%|██████████| 200/200 [00:00<00:00, 267.80it/s]


### Out-of-control

In [13]:
p = 200
w = 100
DeltaNorm = 10 # the norm of shift for Type2
ShiftType = 'Chessboard'

if ShiftType == 'Chessboard':
    # Chessboard
    shift_matrix = []
    element = [[0, 1, 0, -1], [-1, 0, 1, 0]]
    for i in range(w // 5):
        row_element = element[i % 2]
        row = []
        for j in range(p // 10):
            row += [row_element[j % 4]] * 10
        shift_matrix += [row] * 5
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sparse':
    # Sparse
    shift_matrix = np.zeros([w, p])
    shift_matrix[8:13, 18:23] = np.ones([5, 5])

    shift_matrix = np.zeros([w, p])
    for i in range(99):
        shift_matrix[i:(i+2), (2*i):(2*i+2)] = np.ones([2,2])

    shift_matrix = np.zeros([w, p])
    for i in range(25, 80):
        j = int((i**2) * 100 / 80**2)
        shift_matrix[i:(i+3), j:(j+3)] = np.ones([3,3])

    u1 = np.linspace(1, -1, w)
    v1 = np.linspace(1, 0, p)
    shift_matrix = np.matmul(u1.reshape([w, 1]), v1.reshape([1, p]))
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(row)':
    # Sine (row)
    shift_matrix = np.zeros([w, p])
    for i in range(30):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 2 / 20)
    for i in range(30, 60):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 4 / 20)
    for i in range(60, 100):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 6 / 20)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(col)':
    # Sine (column)
    shift_matrix = np.zeros([w, p])
    for i in range(60):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 2 / 10)
    for i in range(60, 120):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 4 / 10)
    for i in range(120, 200):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 6 / 10)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Ring':
    # Ring
    shift_matrix = np.zeros([w, p])
    for i in range(w):
        for j in range(p):
            if int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 <= 3:
                shift_matrix[i, j] = 1
            elif int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 >= 8:
                shift_matrix[i, j] = -1
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)

In [19]:
seedid = 19940402
np.random.seed(seedid)
TrnLen = 10 #30000
SeqLen = 5
ARL0 = 1000

ControlType = 'OutControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'

TagN = (ControlType,DataType,CovType,ShiftType,'Normal','phi',str(phi),'rho',str(rho),'DeltaNorm',str(DeltaNorm))
TagNJoin = ''.join(TagN)
TagE = (ControlType,DataType,CovType,ShiftType,'Exponential','phi',str(phi),'rho',str(rho),'DeltaNorm',str(DeltaNorm))
TagEJoin = ''.join(TagE)
print(CovType)
Covw = getCov(rho,w,CovType)
Covp = getCov(rho,p,CovType)

trnN = []
trnE = []
for n in tqdm(range(SeqLen)):
    trnN_seq = []
    trnE_seq = []
    for i in range(TrnLen):
        eps = matrix_normal.rvs(mean=np.zeros([w,p]), rowcov=Covw, colcov=Covp)
        trnN_seq.append(m + eps)
        trnE_seq.append(m + (-1 * np.log(1 - norm.cdf(eps))))
    trnN.append(trnN_seq)
    trnE.append(trnE_seq)

trnN = np.array(trnN)
trnE = np.array(trnE)

np.save("data/out-of-control/Trn"+TagNJoin+".npy", trnN)
np.save("data/out-of-control/Trn"+TagEJoin+".npy", trnE)    

TriDiagonal


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 25.52it/s]


Exponential


100%|██████████| 5/5 [00:00<00:00, 26.33it/s]


In [20]:
trnN.shape

(5, 10, 100, 200)

# DFIM

## Functions

In [33]:
def get_Gamma0(rho, p, type=1):
    Sigma = np.eye(p)
    if type == 1:
        tmp = np.ones(p-1)*rho
        Sigma = Sigma+np.diag(tmp, k = 1)+np.diag(tmp, k = -1)
    elif type == 2:
        for d in range(1, p-1):
            tmp = np.ones(p-d)*(rho**d)
            Sigma = Sigma+np.diag(tmp, k = d)+np.diag(tmp, k = -d)
    return Sigma

In [45]:
def weight(t):
    g = -24 + 150*t - 150*(t**2)
    return g

# Define the equation of ARL as a function, so that it can be solved using bisection method.
def search_func(my_K, target_ARL0, omega2, line, side=1):
    term = 2 * my_K * (line + 1.166 * np.sqrt(omega2)) /omega2
    f = omega2 / (2 * my_K**2) * (np.exp(term) - 1 - term) - side * target_ARL0
    return f

In [35]:
def get_Omega2(in_data, batch_size):
    N_in = len(in_data)
    overall_sum = 0
    for i in range(N_in - batch_size + 1):
        summation = 0
        b_mean = np.mean(in_data[i:i + batch_size])
        for k in range(1, batch_size + 1):
            summation += weight(k / batch_size) * k ** 2 * (np.mean(in_data[i:i + k]) - b_mean) ** 2 / batch_size
        overall_sum += summation / batch_size
    omega_square = overall_sum / (N_in - batch_size + 1)
    return omega_square

In [36]:
def get_H(omega_square, big_K, target_ARL0, side=1):
    my_h_const = 10
    my_s_const = 1

    while search_func(big_K, target_ARL0, omega_square, my_h_const*np.sqrt(omega_square), side) < 0:
        my_h_const = my_h_const * 2

    while search_func(big_K, target_ARL0, omega_square, my_s_const*np.sqrt(omega_square), side) > 0:
        my_s_const = my_s_const / 2

    high = my_h_const * np.sqrt(omega_square)
    low = my_s_const * np.sqrt(omega_square)

    while high - low > 0.1:
        mid = (high + low) / 2
        test_value = search_func(big_K, target_ARL0, omega_square, mid, side)
        if test_value > 0:
            high = mid
        else:
            low = mid
    control_limit = (high + low) / 2
    return control_limit

In [41]:
def DFTC_setup(in_data, batch_size=50, small_k=0.01, target_arl0=5000, side=1):
    omega_square = get_Omega2(in_data, batch_size)
    print('omega_square: ', omega_square)
    big_K = small_k * np.std(in_data)
    control_limit = get_H(omega_square, big_K, target_arl0, side)
    return np.mean(in_data), big_K, control_limit, omega_square

In [38]:
def DFTC_monitor(data, mu, big_K, control_limit, side=1, return_cusum=False, datetime=[]):
    N = len(data)
    cusum = 0
    cusum_m = 0
    cusum_list = []
    cusum_m_list = []
    for i in range(N):
        cusum = max(0, cusum + (data[i] - mu) - big_K)
        cusum_m = max(0, cusum_m - (data[i] - mu) - big_K)
        cusum_list.append(cusum)
        cusum_m_list.append(cusum_m)
    out_index1 = np.where(np.array(cusum_list) > control_limit)
    out_index2 = np.where(np.array(cusum_m_list) > control_limit)
    if len(out_index1[0]) > 0:
        rl1 = np.min(out_index1[0])
    else:
        rl1 = N
    if len(out_index2[0]) > 0:
        rl2 = np.min(out_index2[0])
    else:
        rl2 = N
    
    if return_cusum and side==1:
        return cusum_list
    if side == 1:
        return rl1 + 1
    else:
        return np.min([rl1, rl2]) + 1

In [67]:
from sklearn.decomposition import TruncatedSVD
class DFIM2:
    def __init__(self, target_arl0 = 1000, side=1, window_size=5, skip_row=0):
        self.target_arl0 = target_arl0
        self.side = side
        self.window_size = window_size
        self.skip_row = skip_row
    
    def setup(self, data_in, target_value=None, rank=1, batch_size=50, small_k=0.01):
        col_num = data_in.shape[1]
        if target_value:
            self.M0 = target_value * np.ones([self.window_size, col_num])
        else:
            self.M0 = np.zeros([self.window_size, col_num])
            for i in np.arange(0, data_in.shape[0] - self.window_size + 1, self.skip_row + 1):
                self.M0 += data_in[i:(i+self.window_size)]
            self.M0 /= (data_in.shape[0] - self.window_size + 1) // (self.skip_row + 1)

        self.u0, self.s0, self.vt0 = np.linalg.svd(self.M0)
        
        lambdaR = []
        lambdaP = []
        for i in tqdm(np.arange(0, data_in.shape[0] - self.window_size + 1, self.skip_row + 1)):
            Y = data_in[i:(i+self.window_size)]
            # ur, sr, vtr = np.linalg.svd(Y - self.M0)
            svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)
            svd.fit(Y - self.M0)
            lambdaR.append(svd.singular_values_[0])
            s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
            lambdaP.append(s2[0])
        self.lambdaR_bar = np.mean(lambdaR)
        self.lambdaP_bar = np.mean(lambdaP)
        lbd = 0.1*10**5

        self.inv_sigma = np.linalg.inv(np.cov([lambdaR, lambdaP])+lbd*np.eye(2))
        data_in_reduced = np.diagonal(
            np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                self.inv_sigma),
                      np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
        )
#         data_reduced = np.sqrt(data_reduced)
        self.mu, self.big_K, self.control_limit, self.omega_square = DFTC_setup(data_in_reduced, batch_size=batch_size, 
                                                                                 small_k=small_k, 
                                                                                 target_arl0=self.target_arl0, 
                                                                                 side=self.side)
        
    def setup_type2(self, data_in, target_value=None, rank=1, batch_size=50, small_k=0.01):
        maxT = data_in.shape[0]
        row_num = data_in.shape[1]
        col_num = data_in.shape[2]
        if target_value:
            self.M0 = target_value * np.ones([row_num, col_num])
        else:
            self.M0 = np.zeros([row_num, col_num])
            for i in np.arange(maxT):
                self.M0 += data_in[i]
            self.M0 /= maxT

        self.u0, self.s0, self.vt0 = np.linalg.svd(self.M0)
        
        lambdaR = []
        lambdaP = []
        for i in tqdm(np.arange(maxT)):
            Y = data_in[i]
            # ur, sr, vtr = np.linalg.svd(Y - self.M0)
            svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)
            svd.fit(Y - self.M0)
            lambdaR.append(svd.singular_values_[0])
            s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
            lambdaP.append(s2[0])
        self.lambdaR_bar = np.mean(lambdaR)
        self.lambdaP_bar = np.mean(lambdaP)
        self.inv_sigma = np.linalg.inv(np.cov([lambdaR, lambdaP]))
        data_in_reduced = np.diagonal(
            np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                self.inv_sigma),
                      np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
        )
#         data_reduced = np.sqrt(data_reduced)
        self.mu, self.big_K, self.control_limit, self.omega_square = DFTC_setup(data_in_reduced, batch_size=batch_size, 
                                                                                 small_k=small_k, 
                                                                                 target_arl0=self.target_arl0, 
                                                                                 side=self.side)
        
    def load(self, M0, inv_sigma, params):
        self.M0 = M0
        self.u0, self.s0, self.vt0 = np.linalg.svd(self.M0)
        self.inv_sigma = inv_sigma
        self.lambdaR_bar = params[0]
        self.lambdaP_bar = params[1]
        self.mu = params[2]
        self.big_K = params[3]
        self.control_limit = params[4]
        
    def monitor(self, data, rank=1, max_RL=10000, return_cusum=False, show_figure=False, datetime=[]):
        if return_cusum:
            lambdaR = []
            lambdaP = []
            for i in np.arange(0, data.shape[0] - self.window_size + 1, self.skip_row + 1):
                Y = data[i:(i+self.window_size)]
                # ur, sr, vtr = np.linalg.svd(Y - self.M0)
                svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)
                svd.fit(Y - self.M0)
                lambdaR.append(svd.singular_values_[0])
                s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
                lambdaP.append(s2[0])
            data_reduced = np.diagonal(
                np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                    self.inv_sigma),
                          np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
            )
    #         data_reduced = np.sqrt(data_reduced)
            cusum_list = DFTC_monitor(data_reduced, self.mu, self.big_K, self.control_limit, 
                                      side=self.side, return_cusum=return_cusum,
                                      show_figure=show_figure, datetime=datetime)
            return cusum_list
        else:
            col_num = data.shape[1]
            cusum = 0
            self.RL = 0
            i = 0
            while cusum <= self.control_limit and self.RL <= max_RL:
                if i + self.window_size > len(data):
                    return 
                else:
                    Y = data[i:(i+self.window_size)]
                # ur, sr, vtr = np.linalg.svd(Y - self.M0)

                svd = TruncatedSVD(n_components=1, n_iter=7, random_state=42)
                svd.fit(Y - self.M0)
                lambdaR = svd.singular_values_[0]
                
                s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
                lambdaP = s2[0]
                data_reduced = np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                    self.inv_sigma),
                          np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
                cusum = max(0, cusum + (data_reduced - self.mu) - self.big_K)
                self.RL += 1
                i += self.skip_row + 1
    
    def monitor_image(self, M, rank=1, max_RL=10000):
        cusum = 0
        self.RL = 0
        cov_row = getCov(rho,w,CovType)
        cov_col = getCov(rho,p,CovType)
        while cusum <= self.control_limit and self.RL <= max_RL:
            Y = matrix_normal.rvs(mean=self.M0+M, rowcov=cov_row, colcov=cov_col, size=1)
            ur, sr, vtr = np.linalg.svd(Y - self.M0)
            lambdaR = sr[0]
            s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
            lambdaP = s2[0]
            data_reduced = np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                self.inv_sigma),
                      np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
            cusum = max(0, cusum + (data_reduced - self.mu) - self.big_K)
            self.RL += 1
    
    def monitor_exp_image(self, M, rank=1, max_RL=10000):
        cusum = 0
        self.RL = 0
        ROW_NUM, COL_NUM = w, p
        cov_row = getCov(rho,w,CovType)
        cov_col = getCov(rho,p,CovType)
        while cusum <= self.control_limit and self.RL <= max_RL:
            norm_epsilon = matrix_normal.rvs(mean=np.zeros([ROW_NUM, COL_NUM]), rowcov=cov_row, colcov=cov_col)
            Y = M + (-1 * np.log(1 - norm.cdf(norm_epsilon)))
            ur, sr, vtr = np.linalg.svd(Y - self.M0)
            lambdaR = sr[0]
            s2 = np.diagonal(np.matmul(np.matmul(self.u0[:, :rank].T, Y), self.vt0[:rank, :].T))
            lambdaP = s2[0]
            data_reduced = np.matmul(np.matmul(np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]).T, 
                                self.inv_sigma),
                      np.array([lambdaR-self.lambdaR_bar, lambdaP-self.lambdaP_bar]))
            cusum = max(0, cusum + (data_reduced - self.mu) - self.big_K)
            self.RL += 1

## Type 1

In [74]:
seedid = 19940402
np.random.seed(seedid)
p = 200
w = 5
TrnLen = 10 #50000
ARL0=1000
mu0 = 5*np.ones(p)
ControlType = 'InControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'Tridiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,'phi',str(phi),'rho',str(rho))
TagJoin = ''.join(Tag)
data_in = np.load("data/in-control/Trn"+TagJoin+".npy")
data_in.shape

(200, 200)

In [75]:
ControlType = 'OutControl'
DataType = 'Profile'
DistType = 'Normal'
rho = 0.3 
CovType = 'TriDiagonal'
phi = 0.3
Tag = (ControlType,DataType,CovType,DistType,ShiftType,'phi',str(phi),'rho',str(rho),'deltanorm',str(deltanorm))
TagJoin = ''.join(Tag)
print(Tag)
data_out = np.load("data/out-of-control/Trn"+TagJoin+".npy")
data_out.shape

('OutControl', 'Profile', 'TriDiagonal', 'Normal', 'Sparse', 'phi', '0.3', 'rho', '0.3', 'deltanorm', '2')


(5, 1000, 200)

In [76]:
dfim2 = DFIM2(skip_row=0)
dfim2.setup(data_in, target_value=5)
dfim2.control_limit

100%|██████████| 196/196 [00:00<00:00, 627.32it/s]


omega_square:  2.3391819511217277e-07


0.009914843493262543

In [77]:
RL = []
for i in tqdm(range(len(data_out))):
    dfim2.monitor(data_out[i])
    RL.append(dfim2.RL)

100%|██████████| 5/5 [00:00<00:00, 14.05it/s]


In [78]:
np.mean(RL), np.std(RL)

(70.2, 38.81443030626625)

## Type 2

In [91]:
ControlType = 'InControl'
DataType = 'Image'
DistType = 'Normal'
phi = 0.3
rho = 0.3 
Phi = phi * np.eye(p)
m = 5*np.ones([w,p])
CovType = 'TriDiagonal'

TagN = (ControlType,DataType,CovType,'Normal','phi',str(phi),'rho',str(rho))
TagNJoin = ''.join(TagN)

data_in = np.load("data/in-control/Trn"+TagNJoin+".npy")
TrnLen,w,p = data_in.shape
TrnLen,w,p

(200, 100, 200)

In [93]:
data_in = data_in.reshape(TrnLen*w,p)
data_in.shape

(20000, 200)

In [96]:
dfim2 = DFIM2(window_size=w, skip_row=w-1)
dfim2.setup(data_in, target_value=5)
dfim2.control_limit

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:00<00:00, 366.40it/s]

omega_square:  1.5520765747124813e-07


0.008076262628982048

In [105]:
DeltaNorm = 10 # the norm of shift for Type2
ShiftType = 'Chessboard'

if ShiftType == 'Chessboard':
    # Chessboard
    shift_matrix = []
    element = [[0, 1, 0, -1], [-1, 0, 1, 0]]
    for i in range(w // 5):
        row_element = element[i % 2]
        row = []
        for j in range(p // 10):
            row += [row_element[j % 4]] * 10
        shift_matrix += [row] * 5
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sparse':
    # Sparse
    shift_matrix = np.zeros([w, p])
    shift_matrix[8:13, 18:23] = np.ones([5, 5])

    shift_matrix = np.zeros([w, p])
    for i in range(99):
        shift_matrix[i:(i+2), (2*i):(2*i+2)] = np.ones([2,2])

    shift_matrix = np.zeros([w, p])
    for i in range(25, 80):
        j = int((i**2) * 100 / 80**2)
        shift_matrix[i:(i+3), j:(j+3)] = np.ones([3,3])

    u1 = np.linspace(1, -1, w)
    v1 = np.linspace(1, 0, p)
    shift_matrix = np.matmul(u1.reshape([w, 1]), v1.reshape([1, p]))
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(row)':
    # Sine (row)
    shift_matrix = np.zeros([w, p])
    for i in range(30):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 2 / 20)
    for i in range(30, 60):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 4 / 20)
    for i in range(60, 100):
        shift_matrix[i,:] = np.sin(np.arange(0, p) * np.pi * 6 / 20)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Sine(col)':
    # Sine (column)
    shift_matrix = np.zeros([w, p])
    for i in range(60):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 2 / 10)
    for i in range(60, 120):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 4 / 10)
    for i in range(120, 200):
        shift_matrix[:,i] = np.sin(np.arange(0, w) * np.pi * 6 / 10)
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)
elif ShiftType == 'Ring':
    # Ring
    shift_matrix = np.zeros([w, p])
    for i in range(w):
        for j in range(p):
            if int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 <= 3:
                shift_matrix[i, j] = 1
            elif int(np.sqrt((i-50)**2 + (j-100)**2)) % 12 >= 8:
                shift_matrix[i, j] = -1
    shift_matrix = DeltaNorm / np.linalg.norm(shift_matrix) * np.array(shift_matrix)

M_shifted = 5*np.ones([w,p])+shift_matrix

In [106]:
RL = []
for i in tqdm(range(5)): # the full scale is 5
    dfim2.monitor_exp_image(max_RL=10000, M=M_shifted)
    RL.append(dfim2.RL)


100%|██████████| 5/5 [00:00<00:00, 37.64it/s]


In [107]:
np.mean(RL), np.std(RL)

(1.0, 0.0)